In [1]:
import numpy as np
import pandas as pd
import json
import requests
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core import Workspace, Dataset


In [2]:
ws = Workspace.from_config()

In [3]:
dataset = Dataset.get_by_name(ws, name='Churn')
df_pd = dataset.to_pandas_dataframe()

Getting data access token with Assigned Identity (client_id=clientid).


In [4]:
webservice_name = 'churn-classifier'

In [5]:
service = Webservice(ws, webservice_name) 

uri = service.scoring_uri
key =service.get_keys()[0]

print(uri)

sampleSize = round(5/len(df_pd),5)
testdf = df_pd.sample(frac=sampleSize)

X = testdf.drop('Exited',axis=1)
y = testdf.Exited.tolist()


input_data = json.dumps(X.to_json( orient='records')) # Create JSON of sample records    

headers = {'Content-Type':'application/json'}

#for AKS deployment you'd need to the service key in the header as well    
headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ key)} 

resp = requests.post(uri, input_data, headers=headers)
try:
    print("POST to url", uri)
    print("input data:", input_data)
    print("label:", y)
    print("prediction:", resp.text)
except Exception:
    print("Web Service Not Found")

http://e915eb4b-856c-4f78-8e76-cd06c848990e.southcentralus.azurecontainer.io/score
POST to url http://e915eb4b-856c-4f78-8e76-cd06c848990e.southcentralus.azurecontainer.io/score
input data: "[{\"CustomerId\":15683521,\"Surname\":\"Godfrey\",\"CreditScore\":594,\"Geography\":\"Germany\",\"Gender\":\"Male\",\"Age\":28,\"Tenure\":0,\"Balance\":142574.71,\"NumOfProducts\":2,\"HasCrCard\":1,\"IsActiveMember\":0,\"EstimatedSalary\":129084.82},{\"CustomerId\":15766172,\"Surname\":\"Tsao\",\"CreditScore\":541,\"Geography\":\"France\",\"Gender\":\"Male\",\"Age\":34,\"Tenure\":3,\"Balance\":128743.55,\"NumOfProducts\":1,\"HasCrCard\":1,\"IsActiveMember\":0,\"EstimatedSalary\":134851.12},{\"CustomerId\":15723706,\"Surname\":\"Abbott\",\"CreditScore\":573,\"Geography\":\"France\",\"Gender\":\"Female\",\"Age\":33,\"Tenure\":0,\"Balance\":90124.64,\"NumOfProducts\":1,\"HasCrCard\":1,\"IsActiveMember\":0,\"EstimatedSalary\":137476.71},{\"CustomerId\":15773779,\"Surname\":\"Jacka\",\"CreditScore\":593